In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("muchocine")

Using custom data configuration default
Reusing dataset muchocine (/home/lewis/.cache/huggingface/datasets/muchocine/default/1.1.1/3ed5582584cd84ef722606a3d725ef18fd4647d63195fef05c47683e5a056ccd)


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review_body', 'review_summary', 'star_rating'],
        num_rows: 3872
    })
})

In [4]:
dataset["train"][0]

{'review_body': '"May, ¿Quieres ser mi amigo?" es una de esas películas que nos recuerdan que el terror no siempre lleva garras de acero en una mano o una mascara en la cara. El terror y la locura se encuentran mucho más cerca de nosotros, de la realidad, de nuestra pacífica y hasta a veces aburrida monotonía. May funciona bajo el método: la bestia duerme dentro de nosotros. En cada uno de nosotros hay un posible psicópata y nuestra vecina la del segundo puede ocultar un oscuro pasado o una doble vida. Para ello, Lucky McKee, nos narra efectivamente una historia presuntamente cotidiana (que sin embargo, engancha desde el principio) sobre una joven muy parecida a esas otras tantas que pululan a nuestro alrededor. La rarita de la clase, la niña tímida de la tienda de comestibles, esa extraña hermana de nuestro amigo, nuestra prima la del pueblo. Es por esto que nos montamos en un tren de cercanías y no en un tren de alta velocidad con escenas más vistas que las reposiciones de "El Princi

In [5]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    "From: https://github.com/huggingface/notebooks/blob/master/examples/text_classification.ipynb"
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset["train"])

In [32]:
dataset.set_format("pandas")

In [33]:
df = dataset["train"][:]

In [34]:
df.head()

,review_body,review_summary,star_rating
0,STALLONE parecía abocado al olvido tras fracas...,El filme da lo que se espera de el sin floritu...,3
1,"Buena película de Gerardo Olivares, que hace b...","Muy interesante drama social, que hace reflexi...",3
2,Woody Allen es un grande del cine de todos los...,"Desprende tal tufillo a telefilm, que difícilm...",2
3,Qué raro. Porque no hablamos de un blockbuster...,"Una película bonita y discreta, con hallazgos ...",3
4,"Si ya lo decía yo, nada bueno sale del híbrido...","No es una película de Jack Black, sino de Jare...",1


In [35]:
df["star_rating"].value_counts()

3    936
2    695
4    665
5    352
1    256
Name: star_rating, dtype: int64

## Tokenize

In [7]:
from transformers import AutoTokenizer

# model_checkpoint = "bertin-project/bertin-roberta-base-spanish"
model_checkpoint = "BSC-TeMU/roberta-base-bne"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [8]:
tokenizer.vocab_size

50262

In [9]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '</s>',
 'pad_token': '<pad>',
 'cls_token': '<s>',
 'mask_token': '<mask>'}

In [10]:
encoded_str = tokenizer.encode("¡hola, me llamo lewis!")

for token in encoded_str:
    print(token, tokenizer.decode([token]))

0 <s>
1465 ¡
12616 hola
66 ,
503  me
17111  llamo
532  le
19514 wis
55 !
2 </s>


In [11]:
import transformers
transformers.__version__

'4.9.2'

In [12]:
tokenizer("¡hola, me llamo lewis!")

{'input_ids': [0, 1465, 12616, 66, 503, 17111, 532, 19514, 55, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
tokenizer("hola!")

{'input_ids': [0, 12616, 55, 2], 'attention_mask': [1, 1, 1, 1]}

In [14]:
tokenizer.decode(11884)

' administrativo'

In [15]:
# 0 <s>
# 1922 ¡
# 11884 hola
# 16 ,
# 378  me
# 13496  llamo
# 466  le
# 91 w
# 350 is
# 5 !
# 2 </s>

In [16]:
tokenizer.max_len_single_sentence

510

In [18]:
def tokenize_text(examples):
    return tokenizer(examples["review_body"], truncation=True)

In [19]:
dataset.reset_format()

In [20]:
tokenize_text(dataset["train"][:2])

{'input_ids': [[0, 56, 20211, 66, 1486, 5624, 499, 737, 4545, 11962, 347, 405, 313, 3319, 5256, 341, 683, 12757, 341, 344, 3505, 403, 1190, 2690, 47739, 313, 10846, 334, 405, 2266, 375, 405, 1361, 9925, 334, 332, 1611, 68, 951, 3505, 342, 332, 14950, 361, 4090, 1204, 467, 2638, 313, 2070, 66, 313, 332, 2132, 66, 313, 1242, 22471, 342, 796, 320, 1979, 41978, 10887, 47260, 68, 3959, 6481, 1912, 344, 7466, 80, 332, 29322, 31290, 1700, 313, 2070, 68, 1267, 956, 993, 313, 2070, 733, 355, 1652, 37128, 26752, 342, 1242, 15917, 332, 369, 2125, 835, 16250, 355, 12141, 1208, 375, 405, 4098, 1078, 68, 6918, 1577, 66, 6668, 6305, 49343, 4360, 66, 683, 7162, 15432, 405, 1397, 15001, 13271, 473, 336, 580, 2036, 66, 48192, 711, 344, 3549, 63, 588, 405, 2704, 727, 21907, 320, 3319, 1312, 7557, 341, 3522, 38054, 320, 1180, 4933, 68, 859, 27001, 725, 313, 332, 3884, 66, 332, 6249, 24485, 313, 332, 5062, 313, 39352, 2449, 66, 1487, 13739, 9447, 313, 1180, 4545, 66, 1242, 9242, 332, 369, 2650, 68, 547, 38

In [21]:
dataset = dataset["train"].train_test_split()

Loading cached split indices for dataset at /home/lewis/.cache/huggingface/datasets/muchocine/default/1.1.1/3ed5582584cd84ef722606a3d725ef18fd4647d63195fef05c47683e5a056ccd/cache-0849cda84c312499.arrow and /home/lewis/.cache/huggingface/datasets/muchocine/default/1.1.1/3ed5582584cd84ef722606a3d725ef18fd4647d63195fef05c47683e5a056ccd/cache-233b7aca96264eb3.arrow


In [22]:
encoded_dataset = dataset.map(tokenize_text, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [23]:
import torch
from transformers import AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 5
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels).to(device)

Some weights of the model checkpoint at BSC-TeMU/roberta-base-bne were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BSC-TeMU/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weig

In [24]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted") 
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [25]:
from transformers import TrainingArguments

batch_size = 16
logging_steps = len(encoded_dataset["train"]) // batch_size
training_args = TrainingArguments(output_dir="results",
                                  num_train_epochs=5,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1",
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch")

In [26]:
encoded_dataset = encoded_dataset.rename_column("star_rating", "label")

In [27]:
encoded_dataset["train"].unique("label")

[3, 2, 1, 4, 5]

In [28]:
device

device(type='cuda')

In [29]:
from transformers import Trainer

trainer = Trainer(model=model, 
                  args=training_args, 
                  compute_metrics=compute_metrics,
                  train_dataset=encoded_dataset["train"],
                  eval_dataset=encoded_dataset["test"],
                  tokenizer=tokenizer)

trainer.train();

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: review_summary, review_body.
***** Running training *****
  Num examples = 2904
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 455
/home/lewis/miniconda3/envs/hf/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


RuntimeError: transform: failed to synchronize: cudaErrorAssert: device-side assert triggered

In [30]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: review_summary, review_body.
***** Running Evaluation *****
  Num examples = 968
  Batch size = 32


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.